Neste notebook, será implementado o algoritmo de treinamento do Perceptron.

In [71]:
!pip install prettytable

In [72]:
import numpy as np
import pandas as pd
from Perceptron import Perceptron
from Activation import Degree
from prettytable import PrettyTable 

## Datasets

Serão utilizados o 'dataAll.txt' para o treinamento do percepetron e o 'dataHoldout.txt' para validação Holdout em Problema Não-Linearmente Separável.

O arquivo de exemplos utilizado para a experimentação será 'data1.txt', conforme o cálculo do identificador da equipe a seguir:

I. $matA + matB + matC + matD = 17$

II. $17 \bmod 4 = 1$


In [13]:
## Inicialização dos dados
examples = 'data/dataAll.txt'
examplesHoldout = 'data/dataHoldout.txt'
examplesExperimentation = 'data/data1.txt'

nparray = np.fromfile(examples)
exparray = np.fromfile(examplesExperimentation)
hlarray = np.fromfile(examplesHoldout)

nparray, hlarray, exparray

(array([-363.7884,  244.1423,    0.    , ..., -140.147 ,  734.0983,
           0.    ], shape=(3000,)),
 array([-0.29322959, -0.09065359,  1.        , ...,  0.72930655,
         0.93224271,  0.        ], shape=(2400,)),
 array([ 0.13658687, -0.16460088,  0.        , ...,  0.99252852,
         1.06937678,  1.        ], shape=(1800,)))

In [14]:
# Formatando o array de entrada
data = nparray.reshape(1000,3)
dataHoldout = hlarray.reshape(800,3)
dataExperimentation = exparray.reshape(600,3)
data, dataHoldout , dataExperimentation

(array([[-363.7884,  244.1423,    0.    ],
        [ 328.7572, -256.7658,    1.    ],
        [-164.9827,  613.2164,    0.    ],
        ...,
        [ 872.4311,  669.1192,    1.    ],
        [ 822.6894, -702.6489,    1.    ],
        [-140.147 ,  734.0983,    0.    ]], shape=(1000, 3)),
 array([[-0.29322959, -0.09065359,  1.        ],
        [ 0.07988839,  0.21101297,  1.        ],
        [-0.07825563, -0.08083512,  1.        ],
        ...,
        [ 0.65980493,  1.05876739,  0.        ],
        [ 1.09867123,  0.87404891,  0.        ],
        [ 0.72930655,  0.93224271,  0.        ]], shape=(800, 3)),
 array([[ 0.13658687, -0.16460088,  0.        ],
        [-0.0228559 , -0.01094684,  0.        ],
        [-0.02654897,  0.00582183,  0.        ],
        ...,
        [ 0.99007304,  0.8921047 ,  1.        ],
        [ 0.91691441,  1.15684083,  1.        ],
        [ 0.99252852,  1.06937678,  1.        ]], shape=(600, 3)))

## Treinamento para Problema Linearmente Separável

In [76]:
def treinamento_perceptron(data, weight_range=None, learning_rate=None):
    X = data[:, :2]
    y = data[:, 2]
    
    #print(f'X: {X[:5]}\n')
    #print(f'Y: {y[:5]}')

    p= Perceptron()
    activation= Degree()

    if weight_range:
        p.w = np.random.uniform(-weight_range, weight_range, 3)
    if learning_rate:
        p.learning_rate = learning_rate
        
    
    
    x = np.insert(X, 0, 1, axis=1) # inserindo o viés
    
    epoch= 0
    convergence= False
    adjusts = 0
    
    while not convergence:
        convergence = True
        
    
        for i in range(len(X)):
            u = np.dot(x[i], p.w)

            y_predicted = activation.f(u)
            error = y[i] - y_predicted
            
            
            if error != 0:
                convergence = False
                p.w = p.w + p.learning_rate * error * x[i]
                #print(p.w)
                adjusts += 1
            
    
        epoch += 1
    
    #print(f'Convergência em {epoch} épocas')
    #print(f'{adjusts} ajustes feitos')
    return epoch, adjusts

In [77]:
treinamento_perceptron(data)

(4, 55)

## Experimentação

In [164]:

def experimentacao(list_learning_rates, list_weight_range):
    dict_configs = {}
    index = 0
    for learning_rate in list_learning_rates:
        for weight_range in list_weight_range:
            
            list_adjusts = []
            list_epochs = []
            
            for i in range(10):
                
                epoch, adjusts = treinamento_perceptron(dataExperimentation,weight_range=weight_range, learning_rate=learning_rate)
                list_adjusts.append(adjusts)
                list_epochs.append(epoch)
                
            dict_configs[index] = {'learning_rate':learning_rate,
                                   'weight_range': weight_range,
                                   'mean_adjusts': np.round(np.mean(list_adjusts),2), 
                                   'std_adjusts': np.round(np.std(list_adjusts),2),
                                   'min_epoch': np.min(list_epochs)}
            index += 1
    return dict_configs
            

In [165]:
dict_configs = experimentacao(list_learning_rates= [0.4,0.1,0.01],list_weight_range= [100,0.5])

In [166]:
from prettytable import PrettyTable, ALL

# Criar objeto PrettyTable
table = PrettyTable()

# Definir título e cabeçalhos
table.title = "Configurações"
table.field_names = [
    "Taxa de Aprendizado", 
    "Intervalo de Pesos",
    "Quantidade de Ajustes",
    "Menor número de épocas para convergência"
]

  # Adiciona linhas horizontais entre as linhas de dados
table.vertical_char = ' '  # Remove as colunas laterais "|

# Adicionar linhas
for key, value in dict_configs.items():
    table.add_row([
        f"η = {value['learning_rate']}",
        f"(-{value['weight_range']},+{value['weight_range']})",
        f"{value['mean_adjusts']} ± {value['std_adjusts']}",
        f"{value['min_epoch']}"
    ])

# Imprimir tabela
print(table)


+-------------------------------------------------------------------------------------------------------------+
                                                 Configurações                                                 
+---------------------+--------------------+-----------------------+------------------------------------------+
  Taxa de Aprendizado   Intervalo de Pesos   Quantidade de Ajustes   Menor número de épocas para convergência  
+---------------------+--------------------+-----------------------+------------------------------------------+
        η = 0.4            (-100,+100)           320.5 ± 155.21                         4                      
        η = 0.4            (-0.5,+0.5)            47.8 ± 14.01                          12                     
        η = 0.1            (-100,+100)          1347.7 ± 766.02                         25                     
        η = 0.1            (-0.5,+0.5)            30.1 ± 15.69                          3               

## ESSE TEXTO PODE SOFRER ALTERAÇÕES POIS OS PESOS INCIAIS SÂO ALEATORIOS
Ao analisar os resultados obtidos na experimentação,foi observado que a configuração com taxa de aprendizado de 0,01 e intervalo de peso (-0,5, +0,5) apresentou o melhor desempenho. Essa configuração precisou apenas de 1 época e 78,8 ± 37,25 ajustes por treinamento para encontrar os pesos adequados e definir uma reta capaz de separar os dados.

Por outro lado, o pior desempenho foi observado na configuração com taxa de aprendizado de 0,01 e intervalo de peso (-100, +100). E nesse caso, foram necessárias 38 épocas e  13.244.5 ± 5.588.03  ajustes para se obter um resultado satisfatório.

Vale lemrbar que os resultados podem variar entre diferentes execuções do experimento, pois a inicialização dos pesos é feita de forma aleatória. Isso significa que, dependendo dos valores inicializados nos pesos, o modelo pode precisar de mais ou menos ajustes para convergir para uma solução.

## Validação Holdout em Problema Não-Linearmente Separável

In [6]:
Xh = dataHoldout[:, :2]
yh = dataHoldout[:, 2]

In [7]:
np.random.seed(1)
index = np.random.permutation(len(dataHoldout)) # shuffle nos dados
split_index = int(0.7 * len(dataHoldout)) #split no 70/30

In [8]:
train_index = index[:split_index]
test_index = index[split_index:]

train_data = dataHoldout[train_index]
test_data = dataHoldout[test_index]

In [9]:
Xh_train = train_data[:, :2]
yh_train = train_data[:, 2]

Xh_test = test_data[:, :2]
yh_test = test_data[:, 2]


### Treinando o Perceptron (Validação Holdout)

In [10]:
xh_train = np.insert(Xh_train, 0, 1, axis=1)

holdoutAdjusts = 0
max_epochs = 100

for epoch in range(max_epochs):
    index = np.random.permutation(len(xh_train))
    xh_train_permuted = xh_train[index]
    yh_train_permuted = yh_train[index]

    for i in range(len(xh_train)):
        u = np.dot(xh_train_permuted[i], p.w)
        yh_predicted = activation.f(u)
        error = yh_train_permuted[i] - yh_predicted

        if error != 0:
            p.w = p.w + p.learning_rate * error * xh_train_permuted[i]
            holdoutAdjusts += 1

    print(f'Época {epoch + 1} finalizada. Pesos: {p.w}')

print(f'Pesos: {p.w}')
print(f'Total de ajustes: {holdoutAdjusts}')


Época 1 finalizada. Pesos: [ -12.10997318  196.96314424 -222.10449475]
Época 2 finalizada. Pesos: [ -20.60997318  178.79623403 -223.38667824]
Época 3 finalizada. Pesos: [ -26.60997318  162.75295674 -223.19372556]
Época 4 finalizada. Pesos: [ -30.80997318  147.96579611 -222.3015852 ]
Época 5 finalizada. Pesos: [ -34.30997318  133.47797879 -221.30748611]
Época 6 finalizada. Pesos: [ -36.70997318  119.74358647 -220.0343783 ]
Época 7 finalizada. Pesos: [ -38.20997318  106.58601563 -218.60960755]
Época 8 finalizada. Pesos: [ -39.40997318   93.80470173 -217.05861466]
Época 9 finalizada. Pesos: [ -40.10997318   81.48765576 -215.40163141]
Época 10 finalizada. Pesos: [ -40.20997318   69.66979744 -213.66480158]
Época 11 finalizada. Pesos: [ -39.40997318   58.73568573 -211.80078376]
Época 12 finalizada. Pesos: [ -37.40997318   48.9325441  -209.84929258]
Época 13 finalizada. Pesos: [ -34.80997318   39.86269344 -207.83025301]
Época 14 finalizada. Pesos: [ -31.60997318   31.49720617 -205.7773103 ]
É